In [8]:
import foodmood_lists as FM
import functools
import pandas
from pandas.io import sql
import sqlalchemy
import pymysql
import numpy

In [41]:
recipe_data = pandas.read_csv("RAW_recipes.csv").head(50)

In [42]:
def listify(tag_string):
    return [tag.strip("'") for tag in tag_string.strip('][').split(", ")]

In [43]:
tag_lists = recipe_data['tags'].apply(listify)
unique_tags = functools.reduce(lambda a,b: set(a).union(set(b)), tag_lists.to_list())

In [44]:
recipe_data.ingredients = recipe_data.ingredients.apply(listify)
recipe_data.tags = recipe_data.tags.apply(listify)
recipe_data.nutrition = recipe_data.nutrition.apply(listify)
recipe_data.steps = recipe_data.steps.apply(listify)

In [45]:
units = ['tsp', 'tbsp', 'oz', 'cup', 'pint', 'quart', 'gallon', 'mL', 'L', 'dL', 'lb', 'mg', 'g', 'kg']
rand_units = [units[i%14] for i in range(2103719)]

In [46]:
ingredient_table = recipe_data[['id', 'ingredients']].explode('ingredients')
ingredient_table['quantity'] = numpy.random.randint(1,10, ingredient_table.shape[0])
ingredient_table['units'] = rand_units
ingredient_table.columns = ['recipeId', 'foodName', 'quantity', 'units']

ValueError: Length of values does not match length of index

In [47]:
ingredient_table

,id,ingredients,quantity
0,137739,winter squash,5
0,137739,mexican seasoning,9
0,137739,mixed spice,5
0,137739,honey,8
0,137739,butter,8
...,...,...,...
49,108414,egg,4
49,108414,heavy cream,7
49,108414,jalapenos,4
49,108414,sharp cheddar cheese,4


In [48]:
tag_table = recipe_data[['id', 'tags']].explode('tags')
tag_table = tag_table[~tag_table.tags.isin(FM.labels) & (~tag_table.tags.isin(FM.cuisines))]
tag_table.columns = ['recipeId', 'tags']

In [49]:
tag_table

,recipeId,tags
0,137739,60-minutes-or-less
0,137739,vegetables
0,137739,easy
0,137739,fall
0,137739,vegetarian
...,...,...
49,108414,vegetables
49,108414,easy
49,108414,beginner-cook
49,108414,inexpensive


In [50]:
recipe_table = recipe_data[['id', 'name', 'minutes', 'n_steps', 'tags', 'n_ingredients', 'contributor_id']]
recipe_table.columns = ['recipeId', 'recipeName', 'cookTime', 'difficulty', 'cuisine', 'servings', 'authorName']

In [51]:
def find_tags(tags, tag_set):
    tagged = list(set(tags).intersection(tag_set))
    if tagged:
        return tagged[-1]
    else:
        return None

In [52]:
recipe_table.cuisine = recipe_table.cuisine.apply(find_tags, args=(FM.cuisines,))

In [53]:
names = pandas.read_csv('StateNames.csv').Name

In [54]:
def id_toName(authorID):
    return names[authorID%names.size]

In [55]:
recipe_table.authorName = recipe_table.authorName.apply(id_toName)

In [56]:
recipe_table['instructionsLink'] = "https://www.food.com/recipe/" + recipe_table.recipeName.str.replace('\s+', '-').str.cat(recipe_table.recipeId.astype('str'), '-')

In [57]:
recipe_table

,recipeId,recipeName,cookTime,difficulty,cuisine,servings,authorName,instructionsLink
0,137739,arriba baked winter squash mexican style,55,11,mexican,7,Deanna,https://www.food.com/recipe/arriba-baked-winte...
1,31490,a bit different breakfast pizza,30,9,north-american,6,Brody,https://www.food.com/recipe/a-bit-different-br...
2,112140,all in the kitchen chili,130,6,None,13,Mya,https://www.food.com/recipe/all-in-the-kitchen...
3,59389,alouette potatoes,45,11,None,11,Marissa,https://www.food.com/recipe/alouette-potatoes-...
4,44061,amish tomato ketchup for canning,190,5,north-american,8,Hettie,https://www.food.com/recipe/amish-tomato-ketch...
5,5289,apple a day milk shake,0,4,north-american,4,Charlotte,https://www.food.com/recipe/apple-a-day-milk-s...
6,25274,aww marinated olives,15,4,canadian,9,Luke,https://www.food.com/recipe/aww-marinated-oliv...
7,67888,backyard style barbecued ribs,120,10,north-american,22,Claire,https://www.food.com/recipe/backyard-style-bar...
8,70971,bananas 4 ice cream pie,180,8,None,6,Hiram,https://www.food.com/recipe/bananas-4-ice-crea...
9,75452,beat this banana bread,70,12,None,9,Anthony,https://www.food.com/recipe/beat-this-banana-b...


In [60]:
keyFile = open("APIkey.txt", "r")
key = keyFile.read()
keyFile.close()

In [63]:
from google_images_search import GoogleImagesSearch
foodImage = GoogleImagesSearch(key, '017970115376021606908:q3caxr88ame')

In [64]:
def find_image(recipe_name):

    _searchParams = {
        'q': recipe_name,
        'num': 1,
        'fileType': 'jpg|png'
    }
    foodImage.search(search_params=_searchParams)
    if foodImage.results():
        return foodImage.results()[0].url
    else:
        return None

In [69]:
recipe_table['imageUrl'] = recipe_test.recipeName.convert_dtypes().str.replace('\s+', ' ').apply(find_image)

({'q': 'arriba baked winter squash mexican style', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'a bit different breakfast pizza', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'all in the kitchen chili', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'alouette potatoes', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'amish tomato ketchup for canning', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'apple a day milk shake', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'aww marinated olives', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'backyard style barbecued ribs', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'bananas 4 ice cream pie', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'beat this banana bread', 'num': 1, 'fileType': 'jpg|png'}, False, None, None, False)
({'q': 'berry good sandwich sp

In [73]:
mySql_config = {
    "pool_size": 5,
    "max_overflow": 2,
    "pool_timeout": 30,
    "pool_recycle": 1800,
}
mySql_engine = sqlalchemy.create_engine(
                sqlalchemy.engine.url.URL(
                    drivername="mysql+pymysql",
                    username='root',
                    password='cs348',
                    host='127.0.0.1',
                    port='3306',
                    database='lol',
                ),
                **mySql_config
            )

In [76]:
from sqlalchemy.types import Integer
from sqlalchemy.types import String

mySql_connection = mySql_engine.connect()
try:
    recipe_frame = recipe_table.to_sql('recipes', mySql_connection, if_exists='replace', 
                        index=False, chunksize=50, 
                        dtype={'recipeId': Integer(), 'recipeName': String(100), 'cookTime': Integer(), 
                                'difficulty': Integer(), 'cuisine': String(50), 'servings': Integer(),      
                                'authorName': String(50), 'instructionsLink': String(100), 'imageUrl': String(200)})

    ingredient_frame = ingredient_table.to_sql('ingredients', mySql_connection, if_exists='replace',
                        index=False, chunksize=50,
                        dtype={'recipeId': Integer(), 'foodName': String(30), 'qantity': Integer(), 
                                'units': String(10)})

    tag_frame = tag_table.to_sql('tags', mySql_connection, if_exists='replace',
                        index=False, chunksize=50,
                        dtype={'recipeId': String(30), 'tags': String(30)})

except ValueError as vx:
    print(vx)

except Exception as ex:   
    print(ex)

else:
    print("Tables created successfully");   

finally:
    mySql_connection.close()

Tables created successfully
